In [1]:
import wrf_management.utilities as ut
import importlib
importlib.reload(ut);
import wrf_management.project_global_constants as gc
importlib.reload(gc)
import wrf_management.geogrid as geo
import os
import sqlite3 as sq
import pandas as pd
import wrf_management.base_namelists.base_namelists as bn
importlib.reload(bn);
import f90nml

In [2]:
con = sq.connect(gc.PATH_DB)

In [15]:
gc.PATH_DB
pd.read_sql('select * from sqlite_master',con)

,type,name,tbl_name,rootpage,sql
0,table,master_date,master_date,2,"CREATE TABLE ""master_date"" (\n""date"" TIMESTAMP..."
1,index,ix_master_date_date,master_date,3,"CREATE INDEX ""ix_master_date_date""ON ""master_d..."
2,table,press,press,8,"CREATE TABLE ""press"" (\n""date"" TEXT,\n ""i"" IN..."
3,index,ix_press_date,press,9,"CREATE INDEX ""ix_press_date""ON ""press"" (""date"")"
4,table,surf_0,surf_0,14,"CREATE TABLE ""surf_0"" (\n""date"" TEXT,\n ""i"" I..."
5,index,ix_surf_0_date,surf_0,15,"CREATE INDEX ""ix_surf_0_date""ON ""surf_0"" (""date"")"
6,table,surf_1,surf_1,20,"CREATE TABLE ""surf_1"" (\n""date"" TEXT,\n ""i"" I..."
7,index,ix_surf_1_date,surf_1,21,"CREATE INDEX ""ix_surf_1_date""ON ""surf_1"" (""date"")"
8,table,runs_table,runs_table,29,"CREATE TABLE ""runs_table"" (\n""index"" INTEGER,\..."
9,index,ix_runs_table_index,runs_table,30,"CREATE INDEX ""ix_runs_table_index""ON ""runs_tab..."


In [33]:
importlib.reload(ut)
program = 'ungrib_surf'
unique_id = ut.get_unique_id(program)
print(unique_id)
# unique_id = '2019-02-21T17-29-01_753696_ungrib_surface'

2019-02-21T20-27-03_589331_ungrib_surf


In [37]:
importlib.reload(ut)
new_path = ut.get_new_unique_path_for_program(unique_id=unique_id)
print(new_path)
os.makedirs(new_path,exist_ok=True)
# new_path = '/wrk/aliagadi/DONOTREMOVE/wrf_management_data/runs/2018_02_19/2019-02-21T17-29-01_753696_ungrib'

/wrk/aliagadi/DONOTREMOVE/wrf_management_data/runs/2018_02_19/2019-02-21T20-27-03_589331_ungrib_surf


In [35]:
ut.get_tb_from_name(tb_name='run_unique_id')

,unique_id,comments,program,run_name
0,2019-02-21T15-04-27_476148_geogrid,exploring geogrid. didnt work as expected,geogrid,2018_02_19
1,2019-02-21T17-29-01_753696_ungrib_ungrib,first run,ungrib,2018_02_19


In [38]:
date_to_ungrib = '2017-12-15'

In [48]:
rows = {}
for tb in gc.FILE_TYPES.keys():
#     print(tb)
    sql = """
    select * from {tb} where date(date)=date('{dt}')
    """.format(tb=tb,dt=date_to_ungrib)
    rows[tb]=pd.read_sql(sql,con)
    rows[tb]['tb']=tb
df = pd.concat(list(rows.values()))
if df.downloaded.all()!=True: print('not all downloaded')
df=df[df.tb.str.startswith('surf')]

In [49]:
df['tar_path']=df.apply(lambda r: os.path.join(ut.get_tar_path(r.tb),r['name']),
         axis = 1)
df['untar_path']=df.apply(lambda r: new_path,
         axis = 1)
row = df.iloc[0]
df

,date,i,downloaded,name,untarred,tb,tar_path,untar_path
0,2017-12-15 00:00:00,14,1,cdas1.20171215.splgrbf.tar,0,surf_0,/wrk/aliagadi/DONOTREMOVE/wrf_management_data/...,/wrk/aliagadi/DONOTREMOVE/wrf_management_data/...
0,2017-12-15 00:00:00,14,1,cdas1.20171215.sfluxgrbf.tar,0,surf_1,/wrk/aliagadi/DONOTREMOVE/wrf_management_data/...,/wrk/aliagadi/DONOTREMOVE/wrf_management_data/...


In [50]:
import tarfile
for l,row in df.iterrows():
    tfile = tarfile.TarFile(row.tar_path)
    tfile.extractall(row.untar_path)
